In [11]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

In [12]:
from typing import List
from langchain_core.language_models.chat_models import BaseChatModel
from langchain.schema.messages import BaseMessage, AIMessage, HumanMessage
from langchain.schema.output import ChatResult, ChatGeneration
from langchain_core.callbacks import Callbacks
from llmstudio.llm import LLM
from typing import Any, Optional
from langchain.chat_models import ChatOpenAI

class ChatLLMstudio(ChatOpenAI):
    model_id: str
    
    @property
    def _llm_type(self):
        return "LLMstudio"

    def _call(
        self,
        prompt: str,
        **kwargs
    ) -> str:
        print("_CALL")
        llm = LLM(self.model_id)
        response = llm.chat(prompt, **kwargs)
        return response.get('chat_output', '')
    
    def __call__(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> BaseMessage:
        print("CALL")
        print({**kwargs})
        input_text = self._convert_messages_to_text(messages)
        llm = LLM(self.model_id)
        response = llm.chat(input_text, **kwargs)
        return AIMessage(content=response.get('chat_output', ''))

    async def _call_async(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> BaseMessage:
        print("CALL ASYNC")
        input_text = self._convert_messages_to_text(messages)
        llm = LLM(self.model_id)
        response = await llm.async_chat(input_text, **kwargs)
        return AIMessage(content=response.get('chat_output', ''))

    def _generate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
        print("GENERATE")
        input_text = self._convert_messages_to_text(messages)
        llm = LLM(self.model_id)
        response = llm.chat(input_text, **kwargs)
        return self._convert_response_to_chat_result(response)

    async def _agenerate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
        print("AGENERATE")
        input_text = self._convert_messages_to_text(messages)
        llm = LLM(self.model_id)
        response = await llm.async_chat(input_text, **kwargs)
        return self._convert_response_to_chat_result(response)

    def _stream(self, messages: List[BaseMessage], **kwargs):
        input_text = self._convert_messages_to_text(messages)
        llm = LLM(self.model_id)
        for chunk in llm.chat(input_text, is_stream=True, **kwargs):
            yield chunk

    async def _astream(self, messages: List[BaseMessage], **kwargs):
        input_text = self._convert_messages_to_text(messages)
        llm = LLM(self.model_id)
        async for chunk in await llm.async_chat(input_text, is_stream=True, **kwargs):
            yield chunk

    def _convert_messages_to_text(self, messages: List[BaseMessage]) -> str:
        return " ".join([message.content for message in messages if isinstance(message, HumanMessage)])

    def _convert_response_to_chat_result(self, response) -> ChatResult:
        chat_output = response.get('chat_output', '')
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=chat_output))])

In [13]:
# gpt3 = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', callbacks=[MyCustomHandler()])

In [14]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, gpt3, agent=AgentType.OPENAI_MULTI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [15]:
assistant('When does my train depart? My ticket number is 1234')




> Entering new AgentExecutor chain...
CALL
{'functions': [{'name': 'tool_selection', 'description': 'A list of actions to take.', 'parameters': {'title': 'tool_selection', 'description': 'A list of actions to take.', 'type': 'object', 'properties': {'actions': {'title': 'actions', 'type': 'array', 'items': {'title': 'tool_call', 'type': 'object', 'properties': {'action_name': {'title': 'action_name', 'enum': ['get_departure', 'cancel_ticket', 'buy_ticket', 'make_complaint'], 'type': 'string', 'description': 'Name of the action to take. The name provided here should match up with the parameters for the action below.'}, 'action': {'title': 'Action', 'anyOf': [{'title': 'get_departure', 'type': 'object', 'properties': {'ticket_number': {'title': 'Ticket Number', 'type': 'string'}}}, {'title': 'cancel_ticket', 'type': 'object', 'properties': {'ticket_number': {'title': 'Ticket Number', 'type': 'string'}}}, {'title': 'buy_ticket', 'type': 'object', 'properties': {'destination': {'title': 

{'input': 'When does my train depart? My ticket number is 1234',
 'output': '{\n  "actions": [\n    {\n      "action_name": "get_departure",\n      "action": {\n        "ticket_number": "1234"\n      }\n    }\n  ]\n}'}

In [16]:
# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[{"role": "user", "content": 'When does my train depart? My ticket number is 1234'}],
#     function_call="auto",
#     functions=[{'name': 'tool_selection', 'description': 'A list of actions to take.', 'parameters': {'title': 'tool_selection', 'description': 'A list of actions to take.', 'type': 'object', 'properties': {'actions': {'title': 'actions', 'type': 'array', 'items': {'title': 'tool_call', 'type': 'object', 'properties': {'action_name': {'title': 'action_name', 'enum': ['get_departure', 'cancel_ticket', 'buy_ticket', 'make_complaint'], 'type': 'string', 'description': 'Name of the action to take. The name provided here should match up with the parameters for the action below.'}, 'action': {'title': 'Action', 'anyOf': [{'title': 'get_departure', 'type': 'object', 'properties': {'ticket_number': {'title': 'Ticket Number', 'type': 'string'}}}, {'title': 'cancel_ticket', 'type': 'object', 'properties': {'ticket_number': {'title': 'Ticket Number', 'type': 'string'}}}, {'title': 'buy_ticket', 'type': 'object', 'properties': {'destination': {'title': 'Destination', 'type': 'string'}}}, {'title': 'make_complaint', 'type': 'object', 'properties': {'complaint': {'title': 'Complaint', 'type': 'string'}}}]}}, 'required': ['action_name', 'action']}}}, 'required': ['actions']}}],
#     stream=True
#     )

# for chunk in response:
#     print(chunk.choices[0].delta.function_call.arguments)



In [17]:
for chunk in response:
    print(response)

NameError: name 'response' is not defined

In [ ]:
import json
response.choices[0].message.function_call.dict()

/var/folders/yf/kbcv02t933zfz4yczvfqcqvw0000gn/T/ipykernel_31096/1974112764.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response.choices[0].message.function_call.dict()


{'arguments': '{\n  "actions": [\n    {\n      "action_name": "get_departure",\n      "action": {\n        "ticket_number": "1234"\n      }\n    }\n  ]\n}',
 'name': 'tool_selection'}